In [ ]:
pip install sgp4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 22.2 MB/s eta 0:00:00


In [ ]:
from sgp4.api import Satrec

In [ ]:
# Assuming the file is named 'tle_data.txt'
filename = '/content/30sats.txt'

# Read the file
with open(filename, 'r') as file:
    tle_lines = file.readlines()

# Display the content of the file
for line in tle_lines:
    print(line.strip())


ISS (ZARYA)
1 25544U 98067A   23055.36715531  .00017001  00000+0  31285-3 0  9996
2 25544  51.6387 167.3561 0005418  22.9195  99.0673 15.49284681384295
AEROCUBE 12A
1 43556U 18046C   23055.30616910  .00104379  00000+0  13224-2 0  9990
2 43556  51.6307  23.8082 0003431 184.7728 175.3232 15.58641168258270
AEROCUBE 12B
1 43557U 18046D   23054.99615263  .00057067  00000+0  96324-3 0  9999
2 43557  51.6349  36.6393 0003695 192.8285 167.2611 15.51072429258029
ICS-EF (ISS DEB)
1 45265U 98067RJ  23054.83215929  .00287185  44651-4  62451-3 0  9991
2 45265  51.6228 103.6745 0006064  32.6113 327.5269 15.97762668171208
ISS DEB
1 47853U 98067RZ  23054.49616732  .00025833  00000+0  30714-3 0  9997
2 47853  51.6386 154.4032 0000899 253.7952 106.2942 15.60965183111018
MMSATS-1
1 47976U 98067SJ  23055.27168209  .00201639  00000+0  70409-3 0  9996
2 47976  51.6314 123.0888 0004985   1.1015 106.7483 15.88194599109771
CSS (TIANHE)
1 48274U 21035A   23055.28861563  .00016711  00000+0  19393-3 0  9993
2 482

In [ ]:
from google.colab import files
import datetime
import numpy as np
from sgp4.api import Satrec, jday
# from datetime import datetime
from datetime import datetime, timedelta

# Step 1: Upload the file
#uploaded = files.upload()

# Assuming the file is named 'tle_data.txt'
filename = '/content/30sats.txt'

# Step 2: Read TLE data from the file
with open(filename, 'r') as file:
    tle_lines = file.readlines()

# Clean up any blank lines and strip whitespace
tle_lines = [line.strip() for line in tle_lines if line.strip()]

# Ensure we have complete TLE entries (each entry has 3 lines)
assert len(tle_lines) % 3 == 0, "TLE data should be in multiples of 3 lines."

# Function to convert position and velocity to required format
def format_output(time, r, v):
    return (time, *r, *v)

# Step 3: Process each TLE entry
results = []
now = datetime.utcnow()
# jd, fr = jday(now.year, now.month, now.day, now.hour, now.minute, now.second)
# jd, fr = jday(2019, 12, 9, 20, 42, 0)
# jd, fr = jday(2000, 1, 1, 1, 1, 0)
start_time = datetime.utcnow()
def datetime_to_jd(dt):
    # Converts datetime to Julian Date
    # Constants
    J2000 = 2451545.0
    # Calculate Julian Date
    JD = (dt - datetime(2000, 1, 1, 12)).total_seconds() / 86400.0 + J2000
    # Julian Date fraction
    frac = (dt - datetime(dt.year, dt.month, dt.day)).total_seconds() / 86400.0
    return JD, frac

jd, fr = datetime_to_jd(current_time)
# jd, fr = 2458826.5, 0.8625

for i in range(0, len(tle_lines), 3):
    name = tle_lines[i]
    tle_line1 = tle_lines[i + 1]
    tle_line2 = tle_lines[i + 2]

    # print(name,tle_line1,tle_line2)

    # Create satellite object
    satellite = Satrec.twoline2rv(tle_line1, tle_line2)

    # Compute position and velocity
    e, r, v = satellite.sgp4(jd, fr)

    if e == 0:
        # Format time as ISO 8601 string
        time_str = now.isoformat()

        # Append result
        results.append(format_output(time_str, r, v))
    else:
        print(f"Error processing TLE entry starting with: {name}")
        results.append((now.isoformat(), "Error", "Error", "Error", "Error", "Error", "Error"))

# Step 4: Display results
import pandas as pd

# Convert results to DataFrame
df = pd.DataFrame(results, columns=["Time", "L(x)", "L(y)", "L(z)", "V(x)", "V(y)", "V(z)"])

# Display the DataFrame
print(df)


Error processing TLE entry starting with: AEROCUBE 12A
Error processing TLE entry starting with: AEROCUBE 12B
Error processing TLE entry starting with: ICS-EF (ISS DEB)
Error processing TLE entry starting with: MMSATS-1
Error processing TLE entry starting with: KITSUNE
Error processing TLE entry starting with: HSU-SAT1
Error processing TLE entry starting with: D3
Error processing TLE entry starting with: BEAVERCUBE
Error processing TLE entry starting with: CLICK-A
Error processing TLE entry starting with: ISS DEB
Error processing TLE entry starting with: 1998-067UL
Error processing TLE entry starting with: 1998-067UM
Error processing TLE entry starting with: 1998-067UN
Error processing TLE entry starting with: 1998-067UP
Error processing TLE entry starting with: XW-4 (CAS-10)
Error processing TLE entry starting with: MARIO
Error processing TLE entry starting with: NUTSAT
Error processing TLE entry starting with: LORIS
Error processing TLE entry starting with: ORCASAT
Error processing T

In [ ]:
import numpy as np
import pandas as pd
from sgp4.api import Satrec, jday
from datetime import datetime, timedelta
import itertools

# Function to compute position and velocity at specific times
def calculate_position_velocity(satellite, start_time, end_time, interval_minutes):
    current_time = start_time
    results = []

    while current_time <= end_time:
        jd, fr = jday(current_time.year, current_time.month, current_time.day,
                      current_time.hour, current_time.minute, current_time.second)
        e, r, v = satellite.sgp4(jd, fr)

        if e == 0:  # Check if the computation was successful
            time_str = current_time.isoformat()
            results.append([time_str, *r, *v])
        else:
            print(f"Error {e} for time {current_time.isoformat()}")

        current_time += timedelta(minutes=interval_minutes)

    return results

# Load and parse TLE data
def load_tle_data(filename):
    with open(filename, 'r') as file:
        tle_lines = file.readlines()

    tle_lines = [line.strip() for line in tle_lines if line.strip()]
    assert len(tle_lines) % 3 == 0, "TLE data should be in multiples of 3 lines."

    tle_entries = [(tle_lines[i + 1], tle_lines[i + 2]) for i in range(0, len(tle_lines), 3)]
    return tle_entries

# Main processing
def process_tle_data(filename, start_time, end_time, interval_minutes):
    tle_entries = load_tle_data(filename)
    all_results = []

    for tle_line1, tle_line2 in tle_entries:
        satellite = Satrec.twoline2rv(tle_line1, tle_line2)
        results = calculate_position_velocity(satellite, start_time, end_time, interval_minutes)
        all_results.extend(results)

    return all_results

# Define parameters
filename = '/content/30sats.txt'  # Replace with your file name
start_time = datetime.utcnow()  # Replace with your desired start time
end_time = start_time + timedelta(days=1)  # One day later
interval_minutes = 1  # Interval in minutes

# Process and save results
results = process_tle_data(filename, start_time, end_time, interval_minutes)

# Convert results to DataFrame and save to CSV
df = pd.DataFrame(results, columns=["Time", "L(x)", "L(y)", "L(z)", "V(x)", "V(y)", "V(z)"])
df.to_csv('satellite_data.csv', index=False)

print(df.head())


Streaming output truncated to the last 5000 lines.
Error 1 for time 2024-08-23T19:02:02.121944
Error 1 for time 2024-08-23T19:03:02.121944
Error 1 for time 2024-08-23T19:04:02.121944
Error 1 for time 2024-08-23T19:05:02.121944
Error 1 for time 2024-08-23T19:06:02.121944
Error 1 for time 2024-08-23T19:07:02.121944
Error 1 for time 2024-08-23T19:08:02.121944
Error 1 for time 2024-08-23T19:09:02.121944
Error 1 for time 2024-08-23T19:10:02.121944
Error 1 for time 2024-08-23T19:11:02.121944
Error 1 for time 2024-08-23T19:12:02.121944
Error 1 for time 2024-08-23T19:13:02.121944
Error 1 for time 2024-08-23T19:14:02.121944
Error 1 for time 2024-08-23T19:15:02.121944
Error 1 for time 2024-08-23T19:16:02.121944
Error 1 for time 2024-08-23T19:17:02.121944
Error 1 for time 2024-08-23T19:18:02.121944
Error 1 for time 2024-08-23T19:19:02.121944
Error 1 for time 2024-08-23T19:20:02.121944
Error 1 for time 2024-08-23T19:21:02.121944
Error 1 for time 2024-08-23T19:22:02.121944
Error 1 for time 2024-08-

In [ ]:
import numpy as np
import pandas as pd
from sgp4.api import Satrec, jday
from datetime import datetime, timedelta

# Function to compute position and velocity at specific times
def calculate_position_velocity(satellite, start_time, end_time, interval_minutes):
    current_time = start_time
    results = []

    while current_time <= end_time:
        # jd, fr = jday(current_time.year, current_time.month, current_time.day,
                      # current_time.hour, current_time.minute, current_time.second)
        jd, fr = 2458826.5, 0.8625
        try:
            e, r, v = satellite.sgp4(jd, fr)

            if e == 0:  # Check if the computation was successful
                time_str = current_time.isoformat()
                results.append([time_str, *r, *v])
            else:
                # Append error result
                print(f"Error {e} for time {current_time.isoformat()}")
                results.append([current_time.isoformat(), "Error", "Error", "Error", "Error", "Error", "Error"])

        except Exception as ex:
            print(f"Exception {ex} for time {current_time.isoformat()}")
            results.append([current_time.isoformat(), "Error", "Error", "Error", "Error", "Error", "Error"])

        current_time += timedelta(minutes=interval_minutes)

    return results

# Load and parse TLE data
def load_tle_data(filename):
    with open(filename, 'r') as file:
        tle_lines = file.readlines()

    tle_lines = [line.strip() for line in tle_lines if line.strip()]
    assert len(tle_lines) % 3 == 0, "TLE data should be in multiples of 3 lines."

    tle_entries = [(tle_lines[i + 1], tle_lines[i + 2]) for i in range(0, len(tle_lines), 3)]
    return tle_entries

# Main processing
def process_tle_data(filename, start_time, end_time, interval_minutes):
    tle_entries = load_tle_data(filename)
    all_results = []

    for tle_line1, tle_line2 in tle_entries:
        try:
            satellite = Satrec.twoline2rv(tle_line1, tle_line2)
            results = calculate_position_velocity(satellite, start_time, end_time, interval_minutes)
            all_results.extend(results)
        except Exception as ex:
            print(f"Exception {ex} processing TLE with line1: {tle_line1} and line2: {tle_line2}")

    return all_results

# Define parameters
filename = '/content/30sats.txt'  # Replace with your file name
start_time = datetime.utcnow()  # Replace with your desired start time
end_time = start_time + timedelta(days=1)  # One day later
interval_minutes = 1  # Interval in minutes

# Process and save results
results = process_tle_data(filename, start_time, end_time, interval_minutes)

# Convert results to DataFrame and save to CSV
df = pd.DataFrame(results, columns=["Time", "L(x)", "L(y)", "L(z)", "V(x)", "V(y)", "V(z)"])
df.to_csv('satellite_data.csv', index=False)

print(df.head())


Streaming output truncated to the last 5000 lines.
Error 6 for time 2024-08-23T19:11:03.810357
Error 6 for time 2024-08-23T19:12:03.810357
Error 6 for time 2024-08-23T19:13:03.810357
Error 6 for time 2024-08-23T19:14:03.810357
Error 6 for time 2024-08-23T19:15:03.810357
Error 6 for time 2024-08-23T19:16:03.810357
Error 6 for time 2024-08-23T19:17:03.810357
Error 6 for time 2024-08-23T19:18:03.810357
Error 6 for time 2024-08-23T19:19:03.810357
Error 6 for time 2024-08-23T19:20:03.810357
Error 6 for time 2024-08-23T19:21:03.810357
Error 6 for time 2024-08-23T19:22:03.810357
Error 6 for time 2024-08-23T19:23:03.810357
Error 6 for time 2024-08-23T19:24:03.810357
Error 6 for time 2024-08-23T19:25:03.810357
Error 6 for time 2024-08-23T19:26:03.810357
Error 6 for time 2024-08-23T19:27:03.810357
Error 6 for time 2024-08-23T19:28:03.810357
Error 6 for time 2024-08-23T19:29:03.810357
Error 6 for time 2024-08-23T19:30:03.810357
Error 6 for time 2024-08-23T19:31:03.810357
Error 6 for time 2024-08-

In [ ]:
import datetime
import csv
from sgp4.api import Satrec, WGS72

def parse_tle_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    satellites = []
    for i in range(0, len(lines), 3):
        name = lines[i].strip()
        tle_line1 = lines[i + 1].strip()
        tle_line2 = lines[i + 2].strip()

        sat = Satrec.twoline2rv(tle_line1, tle_line2, WGS72)
        satellites.append((name, sat))

    return satellites

def compute_satellite_data(satellites, start_time, end_time, interval_minutes):
    current_time = start_time
    data = []

    while current_time <= end_time:
        for name, sat in satellites:
            # Convert current_time to Julian Date
            jd, fr = datetime_to_jd(current_time)
            e, r, v = sat.sgp4(jd, fr)
            if e == 0:
                # Extract position and velocity
                position = r
                velocity = v
                time_str = current_time.strftime('%Y-%m-%d %H:%M:%S')
                data.append((time_str, *position, *velocity))

        current_time += datetime.timedelta(minutes=interval_minutes)

    return data

def datetime_to_jd(dt):
    # Converts datetime to Julian Date
    # Constants
    J2000 = 2451545.0
    # Calculate Julian Date
    JD = (dt - datetime.datetime(2000, 1, 1, 12)).total_seconds() / 86400.0 + J2000
    # Julian Date fraction
    frac = (dt - datetime.datetime(dt.year, dt.month, dt.day)).total_seconds() / 86400.0
    return JD, frac

def save_data_to_file(data, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Time', 'Lx', 'Ly', 'Lz', 'Vx', 'Vy', 'Vz'])
        for entry in data:
            writer.writerow(entry)

if __name__ == "__main__":
    tle_filename = '/content/30sats.txt'  # Replace with your TLE file
    output_filename = 'satellite_data.csv'

    satellites = parse_tle_file(tle_filename)

    start_time = datetime.datetime.utcnow()
    end_time = start_time + datetime.timedelta(days=1)
    interval_minutes = 1

    data = compute_satellite_data(satellites, start_time, end_time, interval_minutes)
    save_data_to_file(data, output_filename)



AttributeError: 'list' object has no attribute 'head'

In [ ]:
import pandas as pd

In [ ]:
d = pd.read_csv('/content/satellite_data.csv')
print(d.head())

<bound method NDFrame.head of                       Time           Lx           Ly           Lz        Vx  \
0      2024-08-23 06:31:37 -4149.780958 -1155.608598 -5147.990039  3.436510   
1      2024-08-23 06:31:37 -2719.349056 -5185.522167  2962.236900  5.980171   
2      2024-08-23 06:31:37 -2474.611915 -4486.382308 -4280.672464  7.119303   
3      2024-08-23 06:31:37 -4990.429997  1151.289145 -4340.316575  1.410421   
4      2024-08-23 06:31:37 -6879.651631  1912.848896 -3368.489415 -4.022064   
...                    ...          ...          ...          ...       ...   
21610  2024-08-24 06:31:37  4251.231158  3345.569197  3895.769106 -5.783490   
21611  2024-08-24 06:31:37  4251.231158  3345.569197  3895.769106 -5.783490   
21612  2024-08-24 06:31:37 -2229.244460  6165.696269  1419.743435 -5.035029   
21613  2024-08-24 06:31:37  6128.963818 -1273.727845  2397.647234  3.060647   
21614  2024-08-24 06:31:37  2804.678280 -3209.067732 -5036.846299  6.701614   

             Vy      

In [ ]:
def load_tles(filename):
  """
  Reads TLE data from a file.

  Args:
      filename: Path to the file containing TLE data.

  Returns:
      A list of tuples, where each tuple contains the satellite name (line 1)
      and the two TLE lines.
  """
  tles = []
  with open(filename, 'r') as f:
    for line1 in f:
      print(line1)
      line2 = next(f)
      name = line1.split()[1]
      tles.append((name, line1.strip(), line2.strip()))
  return tles

satellite_tles = load_tles("/content/30sats.txt")  # Replace with your file path

ISS (ZARYA)             

2 25544  51.6387 167.3561 0005418  22.9195  99.0673 15.49284681384295

1 43556U 18046C   23055.30616910  .00104379  00000+0  13224-2 0  9990

AEROCUBE 12B            

2 43557  51.6349  36.6393 0003695 192.8285 167.2611 15.51072429258029

1 45265U 98067RJ  23054.83215929  .00287185  44651-4  62451-3 0  9991

ISS DEB                 

2 47853  51.6386 154.4032 0000899 253.7952 106.2942 15.60965183111018

1 47976U 98067SJ  23055.27168209  .00201639  00000+0  70409-3 0  9996

CSS (TIANHE)            

2 48274  41.4748 124.4452 0004617 309.7127 125.3013 15.61382156104170

1 49044U 21066A   23054.72618734  .00016796  00000+0  30944-3 0  9999

FREGAT DEB              

2 49271  51.6374 261.1375 0934130 195.8825 161.0755 11.99623062 77260

1 52148U 98067TK  23055.28557458  .00366316  62592-4  10448-2 0  9991

CSS (WENTIAN)           

2 53239  41.4748 128.2815 0004723 306.6360 182.9603 15.61361178104079

1 53462U 98067UB  23054.70581653  .00523464  11924-3  14805-2 0

IndexError: list index out of range

In [ ]:
from datetime import timedelta, datetime

def process_satellite(name, tle1, tle2):
  """
  Calculates the position and velocity of a satellite at one-minute intervals
  for a day.

  Args:
      name: Name of the satellite.
      tle1: First line of the TLE data.
      tle2: Second line of the TLE data.

  Returns:
      A list of dictionaries, where each dictionary contains the time
      (in ISO format), L(x), L(y), L(z), V(x), V(y), and V(z).
  """
  sat = sgp4.TLE(name, tle1, tle2)
  data = []
  start_time = datetime.utcnow()  # Replace with desired start time

  for i in range(1440):  # 1440 minutes in a day
    time = start_time + timedelta(minutes=i)
    year, month, day, hour, minute, second = time.year, time.month, time.day, time.hour, time.minute, time.second

    # Calculate position and velocity vectors
    position, velocity, _ = sat.propagate(year, month, day, hour, minute, second)

    data.append({
      "time": time.isoformat(),
      "L(x)": position[0],  # X-coordinate
      "L(y)": position[1],  # Y-coordinate
      "L(z)": position[2],  # Z-coordinate
      "V(x)": velocity[0],  # X-velocity
      "V(y)": velocity[1],  # Y-velocity
      "V(z)": velocity[2],  # Z-velocity
    })
  return data

# Process each satellite
for name, tle1, tle2 in satellite_tles:
  satellite_data = process_satellite(name, tle1, tle2)
  # Store or use the satellite_data as needed

ModuleNotFoundError: No module named 'sgp4'

In [ ]:
from google.colab import files
import datetime
import numpy as np
from sgp4.api import Satrec, jday
# from datetime import datetime
from datetime import datetime, timedelta

# Step 1: Upload the file
#uploaded = files.upload()

# Assuming the file is named 'tle_data.txt'
filename = '/content/30sats.txt'

# Step 2: Read TLE data from the file
with open(filename, 'r') as file:
    tle_lines = file.readlines()

# Clean up any blank lines and strip whitespace
tle_lines = [line.strip() for line in tle_lines if line.strip()]

# Ensure we have complete TLE entries (each entry has 3 lines)
assert len(tle_lines) % 3 == 0, "TLE data should be in multiples of 3 lines."

# Function to convert position and velocity to required format
def format_output(time, r, v):
    return (time, *r, *v)

# Step 3: Process each TLE entry
results = []
now = datetime.utcnow()
# jd, fr = jday(now.year, now.month, now.day, now.hour, now.minute, now.second)
# jd, fr = jday(2019, 12, 9, 20, 42, 0)
# jd, fr = jday(2000, 1, 1, 1, 1, 0)
start_time = datetime.utcnow()
def datetime_to_jd(dt):
    # Converts datetime to Julian Date
    # Constants
    J2000 = 2451545.0
    # Calculate Julian Date
    JD = (dt - datetime(2000, 1, 1, 12)).total_seconds() / 86400.0 + J2000
    # Julian Date fraction
    frac = (dt - datetime(dt.year, dt.month, dt.day)).total_seconds() / 86400.0
    return JD, frac

jd, fr = datetime_to_jd(current_time)
# jd, fr = 2458826.5, 0.8625

for i in range(0, len(tle_lines), 3):
    name = tle_lines[i]
    tle_line1 = tle_lines[i + 1]
    tle_line2 = tle_lines[i + 2]

    # print(name,tle_line1,tle_line2)

    # Create satellite object
    satellite = Satrec.twoline2rv(tle_line1, tle_line2)

    # Compute position and velocity
    e, r, v = satellite.sgp4(jd, fr)

    if e == 0:
        # Format time as ISO 8601 string
        time_str = now.isoformat()

        # Append result
        results.append(format_output(time_str, r, v))
    else:
        print(f"Error processing TLE entry starting with: {name}")
        results.append((now.isoformat(), "Error", "Error", "Error", "Error", "Error", "Error"))

# Step 4: Display results
import pandas as pd

# Convert results to DataFrame
df = pd.DataFrame(results, columns=["Time", "L(x)", "L(y)", "L(z)", "V(x)", "V(y)", "V(z)"])

# Display the DataFrame
print(df)

ModuleNotFoundError: No module named 'sgp4'

In [ ]:
pip install sgp4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
from sgp4.api import Satrec, WGS72
from sgp4.api import jday
import datetime

def read_tle_file(file_path):
    """Read TLE data from the given file and return a list of tuples (name, line1, line2)."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    satellites = []
    for i in range(0, len(lines), 3):
        name = lines[i].strip()
        line1 = lines[i + 1].strip()
        line2 = lines[i + 2].strip()
        satellites.append((name, line1, line2))

    return satellites

def get_satellite_positions(satrec, start_time, end_time, interval_minutes):
    """Retrieve satellite positions and velocities from start_time to end_time."""
    times = []
    positions = []
    velocities = []

    current_time = start_time
    while current_time <= end_time:
        jd, fr = jday(2023, current_time.month, current_time.day,
                      current_time.hour, current_time.minute, current_time.second)
        e, r, v = satrec.sgp4(jd, fr)

        if e == 0:  # Success
            times.append(current_time)
            positions.append(r)
            velocities.append(v)
        else :
          print(e)
        #   print("error")
        current_time += datetime.timedelta(minutes=interval_minutes)

    return times, positions, velocities

def write_output(n,file_path, times, positions, velocities):
    """Write satellite positions and velocities to the specified file."""
    with open(file_path, 'a') as file:
        for t, pos, vel in zip(times, positions, velocities):
            time_str = t.strftime('%Y-%m-%d %H:%M:%S')
            pos_str = ' '.join(f'{x:.4f}' for x in pos)
            vel_str = ' '.join(f'{x:.4f}' for x in vel)
            file.write(f'{n},{time_str}, {pos_str}, {vel_str}\n')

def main():
    tle_file = '/content/30sats.txt'  # Input TLE file
    output_file = 'satellite_positions.txt'  # Output file
    interval_minutes = 1  # Time interval in minutes

    # Read TLE data
    satellites = read_tle_file(tle_file)

    # Time range
    start_time = datetime.datetime.utcnow()
    end_time = start_time + datetime.timedelta(days=1)

    # Process each satellite
    for name, line1, line2 in satellites:
        satrec = Satrec.twoline2rv(line1, line2, WGS72)
        times, positions, velocities = get_satellite_positions(satrec, start_time, end_time, interval_minutes)

        # Output to file
        write_output(name,output_file, times, positions, velocities)
        print(f'Processed {name}')

if __name__ == '__main__':
    main()


Streaming output truncated to the last 5000 lines.
Error code 1 for time 2024-08-23 22:28:54.526831
Error code 1 for time 2024-08-23 22:29:54.526831
Error code 1 for time 2024-08-23 22:30:54.526831
Error code 1 for time 2024-08-23 22:31:54.526831
Error code 1 for time 2024-08-23 22:32:54.526831
Error code 1 for time 2024-08-23 22:33:54.526831
Error code 1 for time 2024-08-23 22:34:54.526831
Error code 1 for time 2024-08-23 22:35:54.526831
Error code 1 for time 2024-08-23 22:36:54.526831
Error code 1 for time 2024-08-23 22:37:54.526831
Error code 1 for time 2024-08-23 22:38:54.526831
Error code 1 for time 2024-08-23 22:39:54.526831
Error code 1 for time 2024-08-23 22:40:54.526831
Error code 1 for time 2024-08-23 22:41:54.526831
Error code 1 for time 2024-08-23 22:42:54.526831
Error code 1 for time 2024-08-23 22:43:54.526831
Error code 1 for time 2024-08-23 22:44:54.526831
Error code 1 for time 2024-08-23 22:45:54.526831
Error code 1 for time 2024-08-23 22:46:54.526831
Error code 1 for t

In [ ]:
import numpy as np
from sgp4.api import Satrec, WGS72
from sgp4.api import jday
import datetime

def read_tle_file(file_path):
    """Read TLE data from the given file and return a list of tuples (name, line1, line2)."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    satellites = []
    for i in range(0, len(lines), 3):
        name = lines[i].strip()
        line1 = lines[i + 1].strip()
        line2 = lines[i + 2].strip()
        satellites.append((name, line1, line2))

    return satellites

def tle_epoch_to_datetime(epoch_str):
    """Convert TLE epoch string to a datetime object."""
    year = int(epoch_str[:2])
    day_of_year = float(epoch_str[2:])

    # Assuming TLE is for the 2000s
    year += 2000

    # Convert to a date object
    start_of_year = datetime.datetime(year, 1, 1)
    date = start_of_year + datetime.timedelta(days=day_of_year - 1)

    # Extract fractional day
    fractional_day = day_of_year % 1
    date += datetime.timedelta(days=fractional_day)

    return date

def get_julian_date_and_fractional_day(date):
    """Convert datetime to Julian Date and fractional day."""
    jd = date.toordinal() + 1721424.5
    frac_day = (date.hour + date.minute / 60 + date.second / 3600) / 24.0
    return jd + frac_day, frac_day

def get_satellite_positions(satrec, start_time, end_time, interval_minutes):
    """Retrieve satellite positions and velocities from start_time to end_time."""
    times = []
    positions = []
    velocities = []

    current_time = start_time
    while current_time <= end_time:
        try:
            jd, fr = get_julian_date_and_fractional_day(current_time)
            e, r, v = satrec.sgp4(jd, fr)

            if e == 0:  # Success
                times.append(current_time)
                positions.append(r)
                velocities.append(v)
            else:
                print(f"Error code {e} for time {current_time}")
        except Exception as ex:
            print(f"Exception occurred for time {current_time}: {ex}")

        current_time += datetime.timedelta(minutes=interval_minutes)

    return times, positions, velocities

def write_output(name, file_path, times, positions, velocities):
    """Write satellite positions and velocities to the specified file."""
    with open(file_path, 'a') as file:
        for t, pos, vel in zip(times, positions, velocities):
            time_str = t.strftime('%Y-%m-%d %H:%M:%S')
            pos_str = ' '.join(f'{x:.4f}' for x in pos)
            vel_str = ' '.join(f'{x:.4f}' for x in vel)
            file.write(f'{name},{time_str}, {pos_str}, {vel_str}\n')

def main():
    tle_file = '/content/30sats.txt'  # Input TLE file
    output_file = 'satellite_positions.txt'  # Output file
    interval_minutes = 1  # Time interval in minutes

    # Read TLE data
    satellites = read_tle_file(tle_file)

    # Time range
    start_time = datetime.datetime.utcnow()
    end_time = start_time + datetime.timedelta(days=1)

    # Process each satellite
    for name, line1, line2 in satellites:
        satrec = Satrec.twoline2rv(line1, line2, WGS72)
        times, positions, velocities = get_satellite_positions(satrec, start_time, end_time, interval_minutes)

        # Output to file
        write_output(name, output_file, times, positions, velocities)
        print(f'Processed {name}')

if __name__ == '__main__':
    main()


Streaming output truncated to the last 5000 lines.
Error code 1 for time 2024-08-23 22:36:33.738504
Error code 1 for time 2024-08-23 22:37:33.738504
Error code 1 for time 2024-08-23 22:38:33.738504
Error code 1 for time 2024-08-23 22:39:33.738504
Error code 1 for time 2024-08-23 22:40:33.738504
Error code 1 for time 2024-08-23 22:41:33.738504
Error code 1 for time 2024-08-23 22:42:33.738504
Error code 1 for time 2024-08-23 22:43:33.738504
Error code 1 for time 2024-08-23 22:44:33.738504
Error code 1 for time 2024-08-23 22:45:33.738504
Error code 1 for time 2024-08-23 22:46:33.738504
Error code 1 for time 2024-08-23 22:47:33.738504
Error code 1 for time 2024-08-23 22:48:33.738504
Error code 1 for time 2024-08-23 22:49:33.738504
Error code 1 for time 2024-08-23 22:50:33.738504
Error code 1 for time 2024-08-23 22:51:33.738504
Error code 1 for time 2024-08-23 22:52:33.738504
Error code 1 for time 2024-08-23 22:53:33.738504
Error code 1 for time 2024-08-23 22:54:33.738504
Error code 1 for t

In [ ]:
import numpy as np
from sgp4.api import Satrec, WGS72
from sgp4.api import jday
from datetime import datetime, timedelta
import sys

def read_tle_data(filename):
    """Read TLE data from a file and return a list of tuples (name, line1, line2)."""
    tle_entries = []
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            for i in range(0, len(lines), 3):
                name = lines[i].strip()
                line1 = lines[i + 1].strip()
                line2 = lines[i + 2].strip()
                tle_entries.append((name, line1, line2))
    except Exception as e:
        print(f"Error reading TLE data: {e}", file=sys.stderr)
    return tle_entries

def write_output(filename, results):
    """Write results to a file."""
    try:
        with open(filename, 'w') as file:
            for result in results:
                file.write(result + '\n')
    except Exception as e:
        print(f"Error writing output data: {e}", file=sys.stderr)

def convert_epoch_to_jdfr(epoch_year, epoch_day, time):
    """Convert TLE epoch year and day of year to Julian Date and Fractional Day."""
    # Convert the epoch year and day of year to a datetime object
    year = 2000 + epoch_year  # assuming the year is in 2000+ format
    date = datetime(year, 1, 1) + timedelta(days=epoch_day - 1)
    # Add time (hours, minutes, seconds) to the datetime
    date += timedelta(hours=time.hour, minutes=time.minute, seconds=time.second)

    # Convert datetime to Julian Date and Fractional Day
    jd, fr = jday(date.year, date.month, date.day,
                  date.hour, date.minute, date.second)
    return jd, fr

def process_tle_data(tle_entries, start_time, end_time, interval_minutes):
    """Process TLE data and return satellite positions and velocities."""
    results = []
    interval = timedelta(minutes=interval_minutes)

    for name, line1, line2 in tle_entries:
        try:
            satellite = Satrec.twoline2rv(line1, line2, WGS72)
        except Exception as e:
            print(f"Error creating satellite object for {name}: {e}", file=sys.stderr)
            continue

        # Extract TLE epoch year and day of year
        try:
            epoch_year = int(line1[18:20])
            epoch_day_str = line2[20:32].strip()
            # Clean up epoch_day_str to remove any unwanted characters
            epoch_day_str = epoch_day_str.replace(' ', '')  # Remove any spaces
            epoch_day = float(epoch_day_str)
        except ValueError as e:
            print(f"Error extracting epoch data from TLE for {name}: {e}", file=sys.stderr)
            continue

        current_time = start_time
        while current_time <= end_time:
            # Convert the TLE epoch to JD and FR
            try:
                jd, fr = convert_epoch_to_jdfr(epoch_year, epoch_day, current_time)
                e, r, v = satellite.sgp4(jd, fr)

                if e == 0:
                    position = ','.join(f"{coord:.4f}" for coord in r)
                    velocity = ','.join(f"{coord:.4f}" for coord in v)
                    result = f"{current_time.isoformat()},{position},{velocity}"
                    results.append(result)
                else:
                    print(f"Error propagating satellite {name} at {current_time}: Error code {e}", file=sys.stderr)
            except Exception as e:
                print(f"Error during propagation for {name} at {current_time}: {e}", file=sys.stderr)

            current_time += interval

    return results


def main():
    input_filename = '/content/30sats.txt'
    output_filename = 'satellite_positions.csv'
    start_time = datetime(2023, 8, 20, 0, 0, 0)  # Adjust the start time as needed
    end_time = start_time + timedelta(days=1)
    interval_minutes = 1

    tle_entries = read_tle_data(input_filename)
    results = process_tle_data(tle_entries, start_time, end_time, interval_minutes)
    write_output(output_filename, results)

if __name__ == "__main__":
    main()


Error propagating satellite HSU-SAT1 at 2023-08-20 00:00:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:01:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:02:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:03:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:04:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:05:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:06:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:07:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:08:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:09:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:10:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:11:00: Error code 6
Error propagating satellite HSU-SAT1 at 2023-08-20 00:12:00: Error code 6
Error propagating satellite HSU-SAT1 a

In [ ]:
import numpy as np
from sgp4.api import Satrec, WGS72
from sgp4.api import jday
import datetime

def read_tle_file(file_path):
    """Read TLE data from the given file and return a list of tuples (name, line1, line2)."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    satellites = []
    for i in range(0, len(lines), 3):
        name = lines[i].strip()
        line1 = lines[i + 1].strip()
        line2 = lines[i + 2].strip()
        satellites.append((name, line1, line2))

    return satellites

def get_satellite_positions(n,satrec, start_time, end_time, interval_minutes):
    """Retrieve satellite positions and velocities from start_time to end_time."""
    times = []
    positions = []
    velocities = []

    current_time = start_time
    while current_time <= end_time:
        try:
            # Calculate Julian Date and Fractional Day
            jd, fr = jday(2023, current_time.month, current_time.day,
                          current_time.hour, current_time.minute, current_time.second)
            e, r, v = satrec.sgp4(jd, fr)

            if e == 0:  # Success
                times.append(current_time)
                positions.append(r)
                velocities.append(v)
            else:
                print(f"{n} Error code {e} for time {current_time}")
        except Exception as ex:
            print(f"Exception occurred for time {current_time}: {ex}")

        current_time += datetime.timedelta(minutes=interval_minutes)

    return times, positions, velocities

def write_output(name, file_path, times, positions, velocities):
    """Write satellite positions and velocities to the specified file."""
    with open(file_path, 'a') as file:
        for t, pos, vel in zip(times, positions, velocities):
            time_str = t.strftime('%Y-%m-%d %H:%M:%S')
            pos_str = ' '.join(f'{x:.4f}' for x in pos)
            vel_str = ' '.join(f'{x:.4f}' for x in vel)
            file.write(f'{name},{time_str}, {pos_str}, {vel_str}\n')

def main():
    tle_file = '/content/30sats.txt'  # Input TLE file
    output_file = 'satellite_positions.txt'  # Output file
    interval_minutes = 1  # Time interval in minutes

    # Read TLE data
    satellites = read_tle_file(tle_file)

    # Time range
    start_time = datetime.datetime.utcnow()
    end_time = start_time + datetime.timedelta(days=1)

    # Process each satellite
    for name, line1, line2 in satellites:
        try:
            # Create satellite record
            satrec = Satrec.twoline2rv(line1, line2, WGS72)
            times, positions, velocities = get_satellite_positions(name,satrec, start_time, end_time, interval_minutes)

            # Output to file
            write_output(name, output_file, times, positions, velocities)
            print(f'Processed {name}')
        except Exception as e:
            print(f"Failed to process satellite {name}: {e}")

if __name__ == '__main__':
    main()


Streaming output truncated to the last 5000 lines.
MARIO Error code 6 for time 2024-08-23 23:28:40.132985
MARIO Error code 6 for time 2024-08-23 23:29:40.132985
MARIO Error code 6 for time 2024-08-23 23:30:40.132985
MARIO Error code 6 for time 2024-08-23 23:31:40.132985
MARIO Error code 6 for time 2024-08-23 23:32:40.132985
MARIO Error code 6 for time 2024-08-23 23:33:40.132985
MARIO Error code 6 for time 2024-08-23 23:34:40.132985
MARIO Error code 6 for time 2024-08-23 23:35:40.132985
MARIO Error code 6 for time 2024-08-23 23:36:40.132985
MARIO Error code 6 for time 2024-08-23 23:37:40.132985
MARIO Error code 6 for time 2024-08-23 23:38:40.132985
MARIO Error code 6 for time 2024-08-23 23:39:40.132985
MARIO Error code 6 for time 2024-08-23 23:40:40.132985
MARIO Error code 6 for time 2024-08-23 23:41:40.132985
MARIO Error code 6 for time 2024-08-23 23:42:40.132985
MARIO Error code 6 for time 2024-08-23 23:43:40.132985
MARIO Error code 6 for time 2024-08-23 23:44:40.132985
MARIO Error co